In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense

In [2]:
path_train='data/Face_Train'
path_test='data/Face_Test'
img_size=128
num_classes=10
import os
def get_images(path):
    images = []
    labels = []
    k=0
    for subdir,_, files in os.walk(path):
        if subdir==path:
            continue
        for line in files:
            if line.startswith('.'):
                continue
            fields = subdir.strip().split('/')
            file_name=os.path.join(subdir, line)
            label=fields[-1]
            image = cv2.imread(file_name,1)
            image=cv2.resize(image,(img_size,img_size))
            images.append(image)
            labels.append(label)
        k=k+1
    return images, labels

images_train,label_train=get_images(path_train)
images_test,label_test=get_images(path_test)

In [3]:
x_train=np.array(images_train)
print(x_train.shape)
y_train=np.array(label_train)
print(y_train.shape)

(308, 128, 128, 3)
(308,)


In [4]:
x_test=np.array(images_test)
y_test=np.array(label_test)
x_train=x_train/255.0
x_test=x_test/255.0
print(x_test.shape)
print(set(y_train))

(100, 128, 128, 3)
{'9', '1', '3', '5', '6', '7', '8', '0', '2', '4'}


In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
input_shape = (img_size, img_size, 3)

In [7]:
model = Sequential()
#CONV => RELU => POOL
model.add(Conv2D(32, kernel_size=3, padding="same",input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# CONV => RELU => POOL
model.add(Conv2D(64, kernel_size=3, padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Flatten => RELU layers
model.add(Conv2D(128, kernel_size=3, padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Flatten => RELU layers
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("tanh"))

# a softmax classifier
model.add(Dense(num_classes))
model.add(Activation("softmax"))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       7

In [ ]:
batch_size = 32
epochs = 10

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
9/9 [==============================] - 3s 302ms/step - loss: 3.3488 - accuracy: 0.2455 - val_loss: 3.7127 - val_accuracy: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 3s 291ms/step - loss: 1.7818 - accuracy: 0.4079 - val_loss: 4.8345 - val_accuracy: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 3s 315ms/step - loss: 1.2387 - accuracy: 0.6426 - val_loss: 4.5730 - val_accuracy: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 3s 315ms/step - loss: 0.6557 - accuracy: 0.7834 - val_loss: 4.7362 - val_accuracy: 0.0968
Epoch 5/10
9/9 [==============================] - 3s 323ms/step - loss: 0.4325 - accuracy: 0.8448 - val_loss: 4.4012 - val_accuracy: 0.0968
Epoch 6/10
9/9 [==============================] - 3s 327ms/step - loss: 0.2504 - accuracy: 0.9314 - val_loss: 6.8686 - val_accuracy: 0.0968
Epoch 7/10
9/9 [==============================] - 3s 317ms/step - loss: 0.1206 - accuracy: 0.9603 - val_loss: 7.6935 - val_accuracy: 0.0645
Epoch 8/

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])